In [1]:
#data import
from skimage.io import imread
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#gpu import
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

#model imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.metrics import mean_squared_error

In [2]:
#setup GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Num GPUs Available:  1


In [3]:
#image processing function
IMG_H = int(170/3)
IMG_W = int(432/3)
#IMG_D = 3
def preprocessImg(img):
    img = img[70:240,0:432] #170,432,3
    h,w,d = img.shape
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.resize(img,(int(w/3),int(h/3)))
    img = cv2.GaussianBlur(img, (3, 3), 0)
    img = img/255.0
    return img

In [4]:
#read in pandas data
#1. read in full data
steeringDataAll = pd.read_csv('big50night/labeled_data.csv',header=None)
steeringData = steeringDataAll[:11500]
test_data = steeringDataAll[11500:]
#2. drop name of frame column
steeringDataNoImg = steeringData.drop(steeringData.columns[0:2],axis=1)
steeringDataNoImg

,2
0,0.00
1,0.00
2,0.00
3,0.00
4,0.00
...,...
11495,-0.69
11496,-0.15
11497,0.00
11498,0.00


In [5]:
#read in images to array + preprocess
dataDir = 'big50night/frames/'
dataFormat = '.jpg'
dataImages = []
for i in steeringData[0]:
    img = imread('{}{}{}'.format(dataDir,i,dataFormat), as_gray=False)
    img = preprocessImg(img)
    dataImages.append(img)

In [6]:
#images to numpy array
X = np.array(dataImages).reshape(-1,IMG_H,IMG_W,3)
steeringDataNoImg

,2
0,0.00
1,0.00
2,0.00
3,0.00
4,0.00
...,...
11495,-0.69
11496,-0.15
11497,0.00
11498,0.00


In [7]:
#optimizer 
opt = Adam(learning_rate=0.00001)

#model implementation 
model = Sequential()

model.add(Conv2D(36,(3,3),input_shape = X.shape[1:])) 
model.add(Activation('elu')) 
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(42,(3,3))) 
model.add(Activation('elu')) 
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(48,(3,3))) 
model.add(Activation('elu')) 
model.add(MaxPooling2D(pool_size=(1,1)))

model.add(Conv2D(64,(3,3))) 
model.add(Activation('elu')) 
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3))) 
model.add(Activation('elu')) 
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())  
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))

model.add(Dense(1)) 
model.add(Activation('elu'))
#model compil
model.compile(loss='mse', optimizer=opt ,metrics=[mean_squared_error])
hist = model.fit(X,steeringDataNoImg,epochs=300,batch_size=512, validation_split=0.1,verbose=1)

Epoch 1/300
21/21 [==============================] - 5s 115ms/step - loss: 0.4338 - mean_squared_error: 0.4338 - val_loss: 0.1779 - val_mean_squared_error: 0.1779
Epoch 2/300
21/21 [==============================] - ETA: 0s - loss: 0.1767 - mean_squared_error: 0.17 - 1s 69ms/step - loss: 0.1763 - mean_squared_error: 0.1763 - val_loss: 0.1572 - val_mean_squared_error: 0.1572
Epoch 3/300
21/21 [==============================] - 1s 69ms/step - loss: 0.1672 - mean_squared_error: 0.1672 - val_loss: 0.1563 - val_mean_squared_error: 0.1563
Epoch 4/300
21/21 [==============================] - 1s 69ms/step - loss: 0.1643 - mean_squared_error: 0.1643 - val_loss: 0.1559 - val_mean_squared_error: 0.1559
Epoch 5/300
21/21 [==============================] - 1s 70ms/step - loss: 0.1642 - mean_squared_error: 0.1642 - val_loss: 0.1556 - val_mean_squared_error: 0.1556
Epoch 6/300
21/21 [==============================] - 1s 70ms/step - loss: 0.1636 - mean_squared_error: 0.1636 - val_loss: 0.1552 - val_me

21/21 [==============================] - 1s 70ms/step - loss: 0.0265 - mean_squared_error: 0.0265 - val_loss: 0.0475 - val_mean_squared_error: 0.0475
Epoch 97/300
21/21 [==============================] - 1s 69ms/step - loss: 0.0266 - mean_squared_error: 0.0266 - val_loss: 0.0470 - val_mean_squared_error: 0.0470
Epoch 98/300
21/21 [==============================] - 1s 70ms/step - loss: 0.0251 - mean_squared_error: 0.0251 - val_loss: 0.0467 - val_mean_squared_error: 0.0467
Epoch 99/300
21/21 [==============================] - ETA: 0s - loss: 0.0266 - mean_squared_error: 0.0266 ETA: 1s - loss: 0.026 - 1s 70ms/step - loss: 0.0265 - mean_squared_error: 0.0265 - val_loss: 0.0464 - val_mean_squared_error: 0.0464
Epoch 100/300
21/21 [==============================] - 1s 70ms/step - loss: 0.0246 - mean_squared_error: 0.0246 - val_loss: 0.0462 - val_mean_squared_error: 0.0462s - loss: 0.0237 - mean_s - ETA: 0s - loss: 0.0243 - mean_squared_error:  - ETA: 0s - loss: 0.0244 - mean_squared_error: 


Epoch 191/300
21/21 [==============================] - 1s 71ms/step - loss: 0.0141 - mean_squared_error: 0.0141 - val_loss: 0.0195 - val_mean_squared_error: 0.0195
Epoch 192/300
21/21 [==============================] - 2s 74ms/step - loss: 0.0151 - mean_squared_error: 0.0151 - val_loss: 0.0194 - val_mean_squared_error: 0.0194
Epoch 193/300
21/21 [==============================] - 2s 72ms/step - loss: 0.0141 - mean_squared_error: 0.0141 - val_loss: 0.0199 - val_mean_squared_error: 0.0199
Epoch 194/300
21/21 [==============================] - 2s 72ms/step - loss: 0.0142 - mean_squared_error: 0.0142 - val_loss: 0.0199 - val_mean_squared_error: 0.0199
Epoch 195/300
21/21 [==============================] - 1s 71ms/step - loss: 0.0155 - mean_squared_error: 0.0155 - val_loss: 0.0193 - val_mean_squared_error: 0.0193
Epoch 196/300
21/21 [==============================] - 1s 70ms/step - loss: 0.0145 - mean_squared_error: 0.0145 - val_loss: 0.0198 - val_mean_squared_error: 0.0198
Epoch 197/300
21

21/21 [==============================] - 2s 72ms/step - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.0190 - val_mean_squared_error: 0.0190ean_squared_error: 0.01
Epoch 241/300
21/21 [==============================] - 2s 72ms/step - loss: 0.0131 - mean_squared_error: 0.0131 - val_loss: 0.0189 - val_mean_squared_error: 0.0189
Epoch 242/300
21/21 [==============================] - 2s 72ms/step - loss: 0.0129 - mean_squared_error: 0.0129 - val_loss: 0.0187 - val_mean_squared_error: 0.0187
Epoch 243/300
21/21 [==============================] - 1s 71ms/step - loss: 0.0125 - mean_squared_error: 0.0125 - val_loss: 0.0183 - val_mean_squared_error: 0.0183
Epoch 244/300
21/21 [==============================] - 1s 70ms/step - loss: 0.0130 - mean_squared_error: 0.0130 - val_loss: 0.0191 - val_mean_squared_error: 0.0191
Epoch 245/300
21/21 [==============================] - 2s 73ms/step - loss: 0.0127 - mean_squared_error: 0.0127 - val_loss: 0.0187 - val_mean_squared_error: 0.0187
Epoch 2

In [ ]:
#plotting results
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.legend(['training', 'validation'])
plt.ylabel('MSE score')
plt.xlabel('Epoch')
plt.title('loss over time')

In [ ]:
#preprocessing of images for prediction process
def preprocessPredImg(img):
    img = img[70:240,0:432] #170,432,3
    h,w,d = img.shape
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.resize(img,(int(w/3),int(h/3)))
    img = cv2.GaussianBlur(img, (3, 3), 0)
    img = img/255.0
    return img.reshape(-1,IMG_H,IMG_W,3)

In [ ]:
#predicting and comparing to actual values
inc = 11500
for i in test_data[0]:
    testImage = imread('{}{}{}'.format(dataDir,i,dataFormat), as_gray=False)
    p = model.predict([preprocessPredImg(testImage)])
    print("Number: {} Prediction value: {:.3f} Actual Value: {}".format(inc,p[0][0],test_data[2][inc]))
    inc+=1

In [ ]:
#saving model
model.save('mynight50_300.h5')